# Welcome to Agentic AI

## What is Agentic AI?

Agentic AI systems are AI models that can:
- **Think** - Reason about problems
- **Act** - Use tools to solve tasks
- **Adapt** - Make decisions based on context

Today we'll build an AI agent from scratch, starting simple and adding capabilities step by step.

## Step 1: Setup Environment

First, let's import necessary libraries and load our API keys.

In [17]:
from dotenv import load_dotenv

load_dotenv()
print("✅ Environment loaded!")

✅ Environment loaded!


## Step 2: Create a Basic LLM (No Tools Yet)

Let's start with just a language model - no special abilities, just text generation.

In [18]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1
)

print("✅ LLM initialized!")

✅ LLM initialized!


### Test the Basic LLM

Let's see what happens when we ask it to do math without any tools.

In [20]:
response = model.invoke("What is 218039812921+932908921?")
print(response.content)

To calculate the sum, I'll add the two numbers:

218,039,812,921 + 932,908,921 =

The result is: 218,972,721,842


**Notice:** The LLM might get this right by chance, but it's just guessing. It doesn't have a calculator!

## Step 3: Adding Our First Tool - Calculator

Let's give our agent a **calculator tool** so it can do math reliably.

A tool is just a Python function with:
- A clear description
- A specific purpose

In [21]:
from langchain.tools import tool

@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    try:
        result = eval(expression)
        return str(result)
    except:
        return "Error in calculation"

print("✅ Calculator tool created!")

✅ Calculator tool created!


### Create Agent with Calculator

In [22]:
from langchain.agents import create_agent

tools = [calc]
agent_with_calc = create_agent(model, tools=tools)

print("✅ Agent created with 1 tool: Calculator")

✅ Agent created with 1 tool: Calculator


### Demo: Calculator in Action

In [23]:
from pprint import pprint

result = agent_with_calc.invoke(
    {"messages": [{"role": "user", "content": "What is 12 to the power of 3?"}]}
)

print("\nAgent's Response:")
print(result['messages'][-1].content)
pprint(result)


Agent's Response:
The result of 12 to the power of 3 is 1728.
{'messages': [HumanMessage(content='What is 12 to the power of 3?', additional_kwargs={}, response_metadata={}, id='a1bd461b-799f-4dc9-9618-597f549396e2'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '2p7yqxdm1', 'function': {'arguments': '{"expression":"12^3"}', 'name': 'calculator'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 231, 'total_tokens': 247, 'completion_time': 0.037079979, 'completion_tokens_details': None, 'prompt_time': 0.011938263, 'prompt_tokens_details': None, 'queue_time': 0.052607167, 'total_time': 0.049018242}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bdd24-c01b-76b0-ac8c-9409c47c2f85-0', tool_calls=[{'name': 'calculator', 'args': {'expression': '12^3'}, 'id':

## Step 4: Adding Search Capability

Now let's add **internet search** using Tavily, so our agent can find current information.

In [24]:
from langchain_tavily import TavilySearch

search_tool = TavilySearch(max_results=2)
tools = [calc, search_tool]

agent_with_search = create_agent(model, tools=tools)
print("✅ Agent upgraded! Now has: Calculator + Internet Search")

✅ Agent upgraded! Now has: Calculator + Internet Search


### Demo: Search in Action

In [11]:
result = agent_with_search.invoke(
    {"messages": [{"role": "user", "content": "What are the latest trends in AI in 2026?"}]}
)

print("Agent's Response:")
print(result['messages'][-1].content)

Agent's Response:
The latest trends in AI in 2026 include the shift from individual usage to team and workflow orchestration, the evolution of AI from a passive assistant to an active collaborator, and the emergence of multimodal AI that can perceive and act in a world more like humans. Additionally, there will be a focus on enterprise AI adoption, the geopolitics of AI, and AI-driven job displacement. Non-invasive brain-computer interfaces (BCIs) will also be a trend, with ultrasound-based techniques being the most promising approach. Furthermore, cutting-edge reinforcement learning systems will make it possible to automate more of the chip design process, reducing the time and cost to develop a custom chip.


## Step 5: Adding Wikipedia Knowledge

Let's add **Wikipedia** for reliable, detailed information on specific topics.

In [25]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
import wikipedia

wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(wiki_client=wikipedia)
)

tools = [calc, search_tool, wikipedia_tool]

agent_full = create_agent(model, tools=tools)
print("✅ Fully equipped agent! Tools: Calculator + Search + Wikipedia")

✅ Fully equipped agent! Tools: Calculator + Search + Wikipedia


## Step 6: Complex Multi-Tool Task

Now let's see our agent use **multiple tools together** to solve a complex task:
1. Calculate 12³ + 1
2. Identify what's special about this number
3. Find detailed information on Wikipedia

In [26]:
query = (
    "What is the value of 12^3 + 1? "
    "Who is the discoverer of this special number? "
    "Find more about it from wikipedia."
)

print("🚀 Starting multi-tool task...\n")
result = agent_full.invoke(
    {"messages": [{"role": "user", "content": query}]}
)

print("\n" + "="*60)
print("🎯 FINAL RESULT:")
print("="*60)
print(result['messages'][-1].content)

🚀 Starting multi-tool task...


🎯 FINAL RESULT:
The value of 12^3 + 1 is 1729. The discoverer of this special number is Srinivasa Ramanujan, an Indian mathematician. According to Wikipedia, 1729 is known as the Hardy-Ramanujan number or the taxicab number, and it is the smallest number that can be expressed as the sum of two cubes in two different ways: 1729 = 1^3 + 12^3 = 9^3 + 10^3.


## What We've Learned

✨ **Key Takeaways:**

1. **Basic LLM** - Can only generate text based on training
2. **Tools** - Extend capabilities (calculator, search, knowledge bases)
3. **Agents** - Intelligently decide which tools to use and when
4. **Multi-step reasoning** - Break complex tasks into tool-using steps

**The agent can now:**
- 🧮 Calculate accurately
- 🌐 Search the internet
- 📚 Access encyclopedic knowledge
- 🤔 Combine multiple tools to solve complex problems

This is the foundation of **Agentic AI** - autonomous systems that think, act, and adapt!

# Part 2: Multi-Agent Systems

## What are Multi-Agent Systems?

So far we've built a **single agent** with multiple tools. But what if we need:
- **Specialized expertise** - Different agents for different domains
- **Parallel processing** - Multiple agents working simultaneously
- **Complex workflows** - Agents handing off tasks to each other

Let's explore the **4 main multi-agent patterns**:
1. 🔧 **Subagents** - Main agent coordinates specialized subagents
2. 🤝 **Handoffs** - Agents transfer control to each other
3. 📚 **Skills** - Single agent loads specialized knowledge on-demand
4. 🎯 **Router** - Route requests to specialized agents

## Pattern 1: Subagents 🔧

**Concept:** A main agent coordinates specialized subagents as tools.

**Use case:** Customer service bot with specialized agents for billing, technical support, and shipping.

Let's build a simple example with language experts!

In [27]:
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate

python_agent = create_agent(
    model,
    tools=[],
    system_prompt = ("You are a Python expert. Help users with Python-specific questions about syntax, libraries, and best practices.")
)

javascript_agent = create_agent(
    model,
    tools=[],
    system_prompt="You are a JavaScript expert. Help users with JavaScript-specific questions about frameworks, async programming, and DOM manipulation."
)

print("✅ Created specialized subagents: Python & JavaScript experts")

✅ Created specialized subagents: Python & JavaScript experts


### Convert Subagents to Tools

In [28]:
@tool("python_expert", description="Expert in Python programming. Use for Python-specific questions.")
def ask_python_expert(question: str) -> str:
    response = python_agent.invoke({"messages": [{"role": "user", "content": f"{question}"}]})
    return response['messages'][-1].content

@tool("javascript_expert", description="Expert in JavaScript programming. Use for JavaScript-specific questions.")
def ask_js_expert(question: str) -> str:
    response = javascript_agent.invoke({"messages": [{"role": "user", "content": f"{question}"}]})
    return response['messages'][-1].content

print("✅ Subagents wrapped as tools!")

✅ Subagents wrapped as tools!


### Create Main Coordinator Agent

In [29]:
subagent_tools = [ask_python_expert, ask_js_expert]
main_agent = create_agent(model, tools=subagent_tools)

print("✅ Main coordinator agent created with 2 subagent tools")

✅ Main coordinator agent created with 2 subagent tools


### Demo: Subagents Pattern

In [30]:
result = main_agent.invoke({
    "messages": [{
        "role": "user", 
        "content": "What's the difference between list comprehension in Python and array methods in JavaScript?"
    }]
})

print("🎯 Main Agent coordinated both subagents:")
print(result['messages'][-1].content)

🎯 Main Agent coordinated both subagents:
List Comprehensions in Python and Array Methods in JavaScript are both used for transforming and manipulating data in arrays or lists. Here's a comparison of the two:

**List Comprehensions in Python**

List comprehensions are a concise way to create new lists from existing lists or other iterables. They consist of brackets containing an expression followed by a `for` clause, then zero or more `for` or `if` clauses. The result is a new list resulting from evaluating the expression in the context of the `for` and `if` clauses.

Example:
```python
numbers = [1, 2, 3, 4, 5]
double_numbers = [x * 2 for x in numbers]
print(double_numbers)  # [2, 4, 6, 8, 10]
```

**Array Methods in JavaScript**

JavaScript provides several array methods for manipulating and transforming data, including:

* `map()`: Creates a new array with the results of applying a provided function on every element in the calling array.
* `filter()`: Creates a new array with all ele

**✅ Benefits:** 
- Centralized control through main agent
- Specialized expertise in isolated contexts
- Can execute subagents in parallel

**⚠️ Tradeoff:** Extra model calls (results flow back through main agent)

## Pattern 2: Handoffs 🤝

**Concept:** Agents transfer control to each other via tool calls.

**Use case:** Customer support where a general agent can hand off to billing, then billing hands to technical support.

Let's create a coffee shop ordering system with handoffs!

**✅ Benefits:**
- Stateful conversations - context persists across handoffs
- Direct user interaction at each stage
- Efficient for repeat requests (state remembered)

**⚠️ Tradeoff:** Sequential execution - can't parallelize tasks

## Pattern 3: Skills 📚

**Concept:** A single agent loads specialized knowledge/prompts on-demand.

**Use case:** Documentation assistant that loads different API docs as needed.

Think of it as giving one agent access to multiple instruction manuals!

In [32]:
skills_db = {
    "python_syntax": """
    Python Syntax Expert Knowledge:
    - Use def for functions
    - Indentation matters (4 spaces)
    - List comprehensions: [x*2 for x in range(10)]
    - f-strings for formatting: f"Hello {name}"
    - Type hints: def func(x: int) -> str
    """,
    "javascript_async": """
    JavaScript Async Expert Knowledge:
    - Promises: new Promise((resolve, reject) => {...})
    - async/await syntax: async function fetchData() { await... }
    - .then() chaining for sequential operations
    - Promise.all() for parallel operations
    - Error handling: try/catch with async/await
    """,
    "web_security": """
    Web Security Expert Knowledge:
    - Always sanitize user input
    - Use HTTPS everywhere
    - Implement CSRF tokens
    - Avoid SQL injection with parameterized queries
    - Use Content Security Policy (CSP) headers
    """
}

print("✅ Created skill knowledge base with 3 domains")

✅ Created skill knowledge base with 3 domains


### Create Skill-Loading Tools

In [33]:
@tool("load_python_skill")
def load_python_skill() -> str:
    """Load Python syntax knowledge into conversation"""
    return skills_db["python_syntax"]

@tool("load_javascript_skill")
def load_javascript_skill() -> str:
    """Load JavaScript async knowledge into conversation"""
    return skills_db["javascript_async"]

@tool("load_security_skill")
def load_security_skill() -> str:
    """Load web security knowledge into conversation"""
    return skills_db["web_security"]

skill_tools = [load_python_skill, load_javascript_skill, load_security_skill]
print("✅ Skill-loading tools created!")

✅ Skill-loading tools created!


### Create Agent with Skills

In [34]:
skills_agent = create_agent(model, tools=skill_tools)
print("✅ Agent created with access to 3 skills!")

✅ Agent created with access to 3 skills!


### Demo: Skills Pattern

In [35]:
result = skills_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "How do I write a secure async API endpoint in JavaScript?"
    }]
})

print("📚 Agent loaded relevant skills on-demand:")
print(result['messages'][-1].content)

📚 Agent loaded relevant skills on-demand:
To write a secure async API endpoint in JavaScript, you should follow best practices for both async programming and web security. This includes using async/await syntax for readability, handling errors with try/catch blocks, and ensuring that all user input is sanitized to prevent SQL injection or cross-site scripting (XSS) attacks. Additionally, implementing HTTPS, using Content Security Policy (CSP) headers, and protecting against cross-site request forgery (CSRF) are crucial for securing your API endpoint.


**✅ Benefits:**
- Simple architecture - just one agent
- On-demand knowledge loading
- Skills reusable across conversations

**⚠️ Tradeoff:** Context accumulation - all loaded skills stay in memory

## Pattern 4: Router 🎯

**Concept:** A routing layer classifies input and directs it to specialized agents.

**Use case:** Email classification system routing to spam, urgent, or general agents.

The router makes ONE decision, then specialized agents handle the work!

In [36]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel

class RouteQuery(BaseModel):
    category: Literal["math", "science", "history", "general"]

route_prompt = ChatPromptTemplate.from_messages([
    ("system", "Classify the user query into one of these categories: math, science, history, or general."),
    ("human", "{query}")
])

router_chain = route_prompt | model.with_structured_output(RouteQuery)
print("✅ Router created!")

✅ Router created!


### Create Specialized Domain Agents

In [37]:
math_agent = create_agent(model, tools=[calc])
science_agent = create_agent(model, tools=[search_tool, wikipedia_tool])
history_agent = create_agent(model, tools=[wikipedia_tool])
general_agent = create_agent(model, tools=[search_tool])

agents = {
    "math": math_agent,
    "science": science_agent,
    "history": history_agent,
    "general": general_agent
}

print("✅ Created 4 specialized domain agents")

✅ Created 4 specialized domain agents


### Build Router System

In [38]:
def route_query(query: str):
    route_result = router_chain.invoke({"query": query})
    print(f"🎯 Routing to: {route_result.category.upper()} agent")
    
    selected_agent = agents[route_result.category]
    response = selected_agent.invoke({
        "messages": [{"role": "user", "content": query}]
    })
    
    return response['messages'][-1].content

print("✅ Router system ready!")

✅ Router system ready!


### Demo: Router Pattern

In [16]:
test_queries = [
    "What is the square root of 144?",
    "Explain photosynthesis",
    "Who was Napoleon Bonaparte?"
]

for query in test_queries:
    print(f"\n❓ Query: {query}")
    answer = route_query(query)
    print(f"💡 Answer: {answer}\n" + "="*60)


❓ Query: What is the square root of 144?
🎯 Routing to: MATH agent
💡 Answer: The square root of 144 is 12.

❓ Query: Explain photosynthesis
🎯 Routing to: SCIENCE agent
💡 Answer: Photosynthesis is a process by which photopigment-bearing autotrophic organisms, such as plants, algae, and cyanobacteria, convert light energy into chemical energy. This process releases oxygen as a byproduct of water splitting and stores the converted chemical energy within the bonds of intracellular organic compounds, such as carbohydrates. Photosynthesis plays a critical role in producing and maintaining the oxygen content of the Earth's atmosphere and supplies most of the biological energy necessary for complex life on Earth.

❓ Query: Who was Napoleon Bonaparte?
🎯 Routing to: HISTORY agent
💡 Answer: Napoleon Bonaparte was a French general and statesman who rose to prominence during the French Revolution and led a series of military campaigns across Europe during the French Revolutionary and Napoleonic War

**✅ Benefits:**
- Clear separation of routing and execution
- Can parallelize multiple routes
- Easy to add new categories

**⚠️ Tradeoff:** Stateless - each request requires routing overhead

## Pattern Comparison Summary

| Pattern | Best For | Model Calls | Context |
|---------|----------|-------------|---------|
| 🔧 **Subagents** | Parallel execution, isolated contexts | More (4+) | Clean separation |
| 🤝 **Handoffs** | Stateful workflows, direct interaction | Fewer on repeat (2-3) | Accumulates |
| 📚 **Skills** | Simple on-demand knowledge | Fewest (2-3) | Grows with skills |
| 🎯 **Router** | Clear categorization, parallel routing | Moderate (3) | Fresh each time |

### When to use which?

- **Need parallel execution?** → Subagents or Router
- **Need stateful conversations?** → Handoffs or Skills  
- **Need minimal complexity?** → Skills
- **Need centralized control?** → Subagents
- **Need domain isolation?** → Router or Subagents

## 🎓 Final Takeaways

Congratulations! You've mastered:

### Part 1: Single Agent Foundations
✅ Basic LLM capabilities  
✅ Tool integration (Calculator, Search, Wikipedia)  
✅ Multi-tool coordination  
✅ Complex reasoning tasks  

### Part 2: Multi-Agent Patterns
✅ **Subagents** - Coordinated specialists  
✅ **Handoffs** - Stateful transfers  
✅ **Skills** - On-demand knowledge  
✅ **Router** - Smart classification  

### Key Principles

1. **Start Simple** - Single agent with tools often suffices
2. **Add Complexity When Needed** - Use multi-agent for specialization/parallelization
3. **Choose the Right Pattern** - Match your use case to the pattern's strengths
4. **Mix and Match** - Patterns can be combined!

**You're now ready to build production-grade Agentic AI systems! 🚀**